<a href="https://colab.research.google.com/github/LeHoangNhuan/ROBOT_H-N/blob/main/g%E1%BB%8Di%20xe%202%20%C4%91i%E1%BB%83m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-fuzzy

In [2]:
!pip install osmnx networkx folium

In [3]:
!pip install matplotlib

In [4]:
import osmnx as ox
import folium
import random
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact


In [5]:


# Khu vực tìm kiếm
place = " Ho Chi Minh City, Vietnam"

# Tải mạng đường phố (dành cho ô tô)
G = ox.graph_from_place(place, network_type="drive")

# Lấy danh sách tất cả node
nodes, edges = ox.graph_to_gdfs(G)

# Chọn ngẫu nhiên 20 node
sample_nodes = nodes.sample(20, random_state=42)  # random_state để cố định kết quả

# Tạo bản đồ tại trung tâm
center = sample_nodes.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=15)

# Thêm 20 node vào bản đồ
for idx, row in sample_nodes.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        tooltip=f"Node {idx}",
        icon=folium.Icon(color="blue", icon="map-marker", prefix="fa")
    ).add_to(m)

m

/usr/local/lib/python3.12/dist-packages/osmnx/_overpass.py:271: UserWarning: This area is 22 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
/tmp/ipython-input-3598675679.py:14: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = sample_nodes.unary_union.centroid


In [8]:
import folium
from geopy.distance import geodesic
from IPython.display import display, clear_output
import ipywidgets as widgets

# Định nghĩa từ điển các điểm để lựa chọn đường đi
route_points = {
    "Chợ Bến Thành": (10.7721, 106.6983),
    "Dinh Độc Lập": (10.7798, 106.6950),
    "Nhà thờ Đức Bà": (10.7797, 106.6992),
    "Bưu điện trung tâm": (10.7791, 106.6984),
    "Phố đi bộ Nguyễn Huệ": (10.7770, 106.7000),
    "Bảo tàng TP.HCM": (10.7787, 106.6950),
    "Thảo Cầm Viên Sài Gòn": (10.7790, 106.7041),
    "Bitexco Tower": (10.7711, 106.7043),
    "Cầu Mống": (10.7767, 106.7016),
    "Nhà hát TP": (10.7794, 106.6990),
    "Bảo tàng Mỹ thuật": (10.7790, 106.6981),
    "Đường sách Nguyễn Văn Bình": (10.7756, 106.7012),
    "Chợ Lớn Quận 1": (10.7742, 106.6953),
    "Ngã sáu Phù Đổng": (10.7725, 106.6995),
    "Phố Tây Bùi Viện": (10.7667, 106.6994),
    "Nhà thờ Tân Định": (10.7840, 106.6961),
    "Công viên 23/9": (10.7755, 106.6960),
    "Vincom Center": (10.7765, 106.7006),
    "Saigon Centre": (10.7733, 106.7033),
    "Bến Nhà Rồng": (10.7629, 106.6983)
}

# Tạo dropdown cho điểm đi và điểm đến
start_selector = widgets.Dropdown(
    options=list(route_points.keys()),
    value="Chợ Bến Thành",
    description='Điểm đi:',
    disabled=False
)

end_selector = widgets.Dropdown(
    options=list(route_points.keys()),
    value="Thảo Cầm Viên Sài Gòn",
    description='Điểm đến:',
    disabled=False
)

# Tạo dropdown cho loại xe
vehicle_selector = widgets.Dropdown(
    options=['Xe máy', 'Ô tô'],
    value='Ô tô',
    description='Loại xe:',
    disabled=False
)

# Tạo các slider
distance_slider = widgets.FloatSlider(value=5.0, min=1, max=20, step=0.5, description='Khoảng cách:')
traffic_slider = widgets.FloatSlider(value=8.5, min=1, max=10, step=0.5, description='Giao thông:')
weather_slider = widgets.FloatSlider(value=5.0, min=1, max=10, step=0.5, description='Thời tiết:')

# Tạo nút tính toán
compute_button = widgets.Button(description="Tính toán cước phí", button_style='success')
fare_display = widgets.HTML(value="<b>Tổng quãng đường:</b> 0 m &nbsp;&nbsp; <b>Phí cước:</b> 0 VND")

# Hiển thị các widget ban đầu
display(start_selector, end_selector, vehicle_selector, distance_slider, traffic_slider, weather_slider, compute_button, fare_display)

def compute_and_display(b):
    try:
        print("Đang tính toán...")  # Debug message

        # Lấy thông tin từ các widget
        start_name = start_selector.value
        end_name = end_selector.value
        vehicle_type = vehicle_selector.value

        print(f"Điểm đi: {start_name}, Điểm đến: {end_name}, Loại xe: {vehicle_type}")

        start_coords = route_points[start_name]
        end_coords = route_points[end_name]

        print(f"Tọa độ điểm đi: {start_coords}, Tọa độ điểm đến: {end_coords}")

        # Tạo bản đồ
        m = folium.Map(location=start_coords, zoom_start=15)

        # Thêm điểm đi và điểm đến
        folium.Marker(start_coords, popup=f'Điểm đi: {start_name}',
                      icon=folium.Icon(color='green', icon='play')).add_to(m)
        folium.Marker(end_coords, popup=f'Điểm đến: {end_name}',
                      icon=folium.Icon(color='red', icon='stop')).add_to(m)

        # Thêm đường đi (giả lập)
        folium.PolyLine([start_coords, end_coords], color='blue', weight=3, opacity=0.7).add_to(m)

        # Tính toán khoảng cách
        distance = geodesic(start_coords, end_coords).meters
        print(f"Khoảng cách: {distance} meters")

        # Tính phí cước (giả lập)
        base_fare = 10000 + (distance * 0.1) + (traffic_slider.value * 1000) + (weather_slider.value * 500)
        final_fare = base_fare * 0.8 if vehicle_type == 'Xe máy' else base_fare

        fare_display.value = f"<b>Tổng quãng đường:</b> {round(distance,1)} m &nbsp;&nbsp; <b>Phí cước:</b> {round(final_fare,0)} VND"

        # Hiển thị kết quả
        clear_output(wait=True)
        display(start_selector, end_selector, vehicle_selector, distance_slider, traffic_slider, weather_slider, compute_button, fare_display)
        display(m)

        print("Tính toán hoàn tất!")

    except Exception as e:
        print(f"Lỗi xảy ra: {e}")
        import traceback
        traceback.print_exc()

# Gán sự kiện cho nút - QUAN TRỌNG: Đảm bảo dòng này được chạy
compute_button.on_click(compute_and_display)

print("Chọn điểm đi, điểm đến và nhấn 'Tính toán cước phí' để xem kết quả")
print("Đảm bảo nút đã được kết nối với hàm tính toán")

Dropdown(description='Điểm đi:', index=19, options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà thờ Đức Bà', 'Bưu đi…

Dropdown(description='Điểm đến:', index=18, options=('Chợ Bến Thành', 'Dinh Độc Lập', 'Nhà thờ Đức Bà', 'Bưu đ…

Dropdown(description='Loại xe:', options=('Xe máy', 'Ô tô'), value='Xe máy')

FloatSlider(value=5.0, description='Khoảng cách:', max=20.0, min=1.0, step=0.5)

FloatSlider(value=8.5, description='Giao thông:', max=10.0, min=1.0, step=0.5)

FloatSlider(value=5.0, description='Thời tiết:', max=10.0, min=1.0, step=0.5)

Button(button_style='success', description='Tính toán cước phí', style=ButtonStyle())

HTML(value='<b>Tổng quãng đường:</b> 1273.7 m &nbsp;&nbsp; <b>Phí cước:</b> 16902.0 VND')

Tính toán hoàn tất!
